In [1]:
import numpy as np
import os
import glob
import pandas as pd
from statsmodels.sandbox.stats.multicomp import multipletests
import numpy as np
from subprocess import call

In [2]:
dir_pre = "/mnt/hpc/home/xuxinran/DirectSeq/data/zhaolin_240206/240201-zhaolin-RNA-merge/v0.7.2/stability/nano_merge"

output_path = f"{dir_pre}_signal_stqtl.csv"
output_path_all = f"{dir_pre}_stability_SNP.csv"

In [3]:
files = glob.glob(f'{dir_pre}_haplotype_chr*_tmp.csv')
dfs = [pd.read_csv(file) for file in files]
df = pd.concat(dfs, ignore_index=True)
df = df.sort_values(by=['chrom', 'snp_pos_1base'])
df = df.reset_index(drop=True)
df.dropna(subset=['p_value'], inplace=True)
df = df[df['p_value'] != 0]

In [4]:
df

,chrom,strand,snp_pos_1base,rsID,A1,A2,A1_STscore_l,A2_STscore_l,EAF,p_value,beta,SE
0,chr1,+,787399,rs2905055,G,T,"[0.1754788495353134, 0.1754788495353134, 0.175...","[0.1754788495353134, 0.1754788495353134, 0.175...",0.241900,0.128224,1.586009e-03,1.019908e-03
1,chr1,-,879911,rs143853699,T,C,"[0.1754788495353134, 0.1719738030669534, 0.163...","[0.1754788495353134, 0.1754788495353134, 0.175...",0.014420,0.334632,7.784999e-03,7.891834e-03
2,chr1,+,900505,rs28705211,C,G,"[-0.3915318654372688, -0.2095703789888585, -0....","[-0.379749369068438, -0.8964588627480949, 0.17...",0.080810,0.431176,1.061763e-01,1.338320e-01
3,chr1,+,900730,rs3935066,G,A,"[0.1754788495353134, 0.1754788495353134, -0.03...","[-0.379749369068438, -0.8964588627480949, 0.17...",0.293300,0.177263,-1.214090e-01,8.887053e-02
4,chr1,+,900972,rs9697711,T,G,"[-0.0333586030104849, 0.1754788495353134, 0.17...","[-0.379749369068438, -0.8964588627480949, 0.17...",0.082520,0.376728,-7.974626e-02,8.969504e-02
...,...,...,...,...,...,...,...,...,...,...,...,...
19768,chr9,+,140167985,rs28740782,T,A,"[0.1754788495353134, 0.1754788495353134, 0.175...","[0.1754788495353134, 0.1754788495353134, 0.175...",0.476000,0.595243,-5.003678e-03,9.415888e-03
19769,chr9,+,140175422,rs117569577,T,C,"[0.1754788495353134, 0.1754788495353134, 0.175...","[0.1754788495353134, 0.1754788495353134, 0.175...",0.074260,0.063966,-1.084202e-16,5.820156e-17
19770,chr9,+,140176814,rs112239920,A,G,"[0.1754788495353134, 0.1754788495353134, 0.175...","[0.1754788495353134, 0.1754788495353134, 0.175...",0.261400,0.126235,2.569836e-03,1.679175e-03
19771,chr9,+,140446707,rs149264137,A,G,"[0.1754788495353134, 0.1754788495353134, 0.175...","[0.1754788495353134, 0.1754788495353134, 0.175...",0.009709,0.585571,2.205143e-03,4.043688e-03


In [5]:
p_adjusted = multipletests(df["p_value"], method='fdr_bh')
df['fdr'] = p_adjusted[1]
print("pvalue小于0.05的个数：", len(df[df['p_value']<0.05]))
print("pvalue小于0.01的个数：", len(df[df['p_value']<0.01]))
print("fdr小于0.1的个数：", len(df[df['fdr']<0.1]))

pvalue小于0.05的个数： 1999
pvalue小于0.01的个数： 1241
fdr小于0.1的个数： 1086


In [6]:
result_df = df[df['p_value']<0.05]
result_df.to_csv(output_path, index=False)
df.to_csv(output_path_all, index=False)

call(f'rm {dir_pre}_haplotype_chr*_tmp.csv',shell=True)

0